<a href="https://colab.research.google.com/github/MDankloff/ScriptAIRbnb.R/blob/main/COMPAS_data_inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")
import os

# Specify the directory path
directory_path = '//Users/mirthedankloff/Documents/XAI_Bias_Error/COMPAS'

# Traverse the directory and print the names of all files
for dirname, _, filenames in os.walk(directory_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))


LOAD COMPAS data

In [28]:
compas_s = pd.read_csv("/content/final_preprocessed_compas(2).csv")
ofercompas = pd.read_csv("/content/propublica_data_for_fairml.csv")
originalcompas = pd.read_csv("/content/compas-scores-two-years-violent.csv")

In [29]:
target = "Two_yr_Recidivism"
X = ofercompas.drop([target],axis=1)
Y = ofercompas[target]

TRAIN  Model

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, shuffle = True, stratify = Y)

model = RandomForestClassifier()
model.fit(X_train, Y_train)

#predictions
Yhat = model.predict(X_test)
len(Yhat)


1852

COLLECT the errors

In [57]:
predictions = pd.DataFrame()
predictions['predicted class'] = Yhat.tolist()
predictions = predictions.set_index(X_test.index)
predictions['true class'] = Y_test

#accuracy
accuracy = accuracy_score(predictions['true class'], predictions['predicted class'])
print(f"Accuracy: {accuracy:.4f}")

#confusion matrix
cm = confusion_matrix(predictions['true class'], predictions['predicted class'])
print(cm)


Accuracy: 0.6566
[[726 283]
 [353 490]]


Selecting 4 labels (FP, FN, TP, TN)

In [58]:
TP = cm[1,1]
FP = cm[0,1]
FN = cm[1,0]
TN = cm[0,0]

predictions['True Positives'] = TP
predictions['False Positives'] = FP
predictions['False Negatives'] = FN
predictions['True Negatives'] = TN

Scatterplot 4 labels (FP, FN, TP, TN)

PCA 4 labels (FP, FN, TP, TN)